In [2]:
# Define Neo4j connections
from neo4j import GraphDatabase
host = 'bolt://localhost:7687'
user = 'neo4j'
password = 'zeppelin'
driver = GraphDatabase.driver(host,auth=(user, password))

In [3]:
# Import libraries
import pandas as pd

def read_query(query):
    with driver.session() as session:
        result = session.run(query)
        return pd.DataFrame([r.values() for r in result], columns=result.keys())
    
def run_query(query):
    with driver.session() as session:
        session.run(query)

## Import

I have used the GoT dataset more times than I can remember, so I decided to explore the internet and search for new exciting graphs. I stumbled upon this Lord of the Rings dataset made available by José Calvo that we will use in this blog post.

The dataset describes interactions between persons, places, groups, and things (The Ring). When choosing how to model this dataset, I decided to have "main" nodes with two labels, primary label "Node" and secondary label one of the following:

* Person
* Place
* Group
* Thing

In [4]:
# Import nodes
import_nodes_query = """

LOAD CSV WITH HEADERS FROM 
"https://raw.githubusercontent.com/morethanbooks/projects/master/LotR/ontologies/ontology.csv" as row 
FIELDTERMINATOR "\t" 
WITH row, CASE row.type WHEN 'per' THEN 'Person' 
                        WHEN 'gro' THEN 'Group' 
                        WHEN 'thin'THEN 'Thing' 
                        WHEN 'pla' THEN 'Place' 
                        END as label 
CALL apoc.create.nodes(['Node',label], [apoc.map.clean(row,['type','subtype'],[null,""])]) YIELD node 
WITH node, row.subtype as class 
MERGE (c:Class{id:class}) 
MERGE (node)-[:PART_OF]->(c)

"""

run_query(import_nodes_query)

In [5]:
# Import relationships
import_relationships_query = """

UNWIND ['1','2','3'] as book 
LOAD CSV WITH HEADERS FROM 
"https://raw.githubusercontent.com/morethanbooks/projects/master/LotR/tables/networks-id-volume" + book + ".csv" AS row 
MATCH (source:Node{id:coalesce(row.IdSource,row.Source)})
MATCH (target:Node{id:coalesce(row.IdTarget,row.Target)})
CALL apoc.create.relationship(source, "INTERACTS_" + book, 
     {weight:toInteger(row.Weight)}, target) YIELD rel
RETURN distinct true

"""

run_query(import_relationships_query)

## Graph data science

### Cypher projection

The general syntax to use cypher projection is:
<pre>
CALL gds.graph.create.cypher(
    graphName: String,
    nodeQuery: String,
    relationshipQuery: String,
    configuration: Map
)
</pre>
Node query is a cypher statement used to describe the nodes we want to project. It must return the internal ids of the nodes and optionally any of their properties. Relationship query, on the other hand, describes the relationships we want to project. The cypher statement should return internal ids of source and target nodes of relationships, and optionally their type and any of their properties.

In [6]:
def drop_graph(name):
    with driver.session() as session:
        drop_graph_query = """
        CALL gds.graph.drop('{}');
        """.format(name)
        session.run(drop_graph_query)

### Whole graph projection

We will begin with a simple scenario and project the whole graph in memory. Adding the column type in the relationship query allows the data science engine to distinguish between relationship types, which in turn gives us an option to filter relationships when executing algorithms.

In [9]:
whole_graph = """
CALL gds.graph.create.cypher( 'whole-graph', 
    // nodeQuery 
    'MATCH (n) RETURN id(n) AS id', 
    // relationshipQuery 'MATCH (n)-[r]->(m) 
    'MATCH (n)-[r]->(m) RETURN id(n) AS source, id(m) AS target, type(r) as type')
"""
run_query(whole_graph);

As in the previous blog post, we will start with the weakly connected components algorithm. It is used to examine how many islands or disconnected components are there in our graph, which will help us better understand results from other graph algorithms. Also, sometimes we might want to run other graph algorithms only on the largest connected component.

In [11]:
wcc_whole = """

CALL gds.wcc.stream('whole-graph') 
YIELD nodeId, componentId 
RETURN componentId, count(*) as size 
ORDER BY size DESC LIMIT 10

"""
read_query(wcc_whole)

,componentId,size
0,0,86


As there is only one connected component in our graph, we don't have to worry about skewed results from other graph algorithms.

In [13]:
# Drop whole graph
drop_graph('whole-graph')

### Undirected weighted relationships graph

Next, we are going to project an undirected weighted graph. Let's take a look at how does the native projection handle undirected relationships:

* UNDIRECTED: each relationship is projected in both natural and reverse orientation

To produce an undirected relationship with the cypher projection, we project a single relationship in both directions, effectively allowing the graph data science engine to traverse the relationship in both directions. Let's take a look at the following example to gain a better understanding. We create two nodes with a single relationship between them.

<pre>
CREATE (:Test)-[:REL]->(:Test);
</pre>

To project the relationship in both directions, we only have to omit the direction of the relationship in our <code>MATCH</code> statement and that's it. A tiny, but very important detail!

<pre>
MATCH (n:Test)-[r]-(m:Test)
RETURN id(n) as source, id(m) as target;
</pre>

Results

|source |	target |
|-------|----------|
|1565   |	1566   |
|1566   |	1565   |

We will also demonstrate a favorable way to project multiple node labels using a UNION statement in the node query.

In [14]:
undirected_interacts_query = """
CALL gds.graph.create.cypher('undirected_interactions', 
    // nodeQuery 
    'MATCH (n:Person) RETURN id(n) AS id 
     UNION MATCH (n:Thing) 
     RETURN id(n) as id', 
    // relationshipQuery (notice no direction on relationship) 
    'MATCH (n)-[r:INTERACTS_1|:INTERACTS_2|:INTERACTS_3]-(m) 
     RETURN id(n) AS source, id(m) AS target, type(r) as type, r.weight as weight')
"""
run_query(undirected_interacts_query)

### Random walk algorithm

To stray away from the common graph algorithms like PageRank or Louvain, let's use the Random Walk algorithm. In essence, it mimics how a drunk person would traverse our graph. It is commonly used in the node2vec algorithms. We define Frodo as the start node and then walk five random steps twice.

In [15]:
random_walk = """

MATCH (n:Node{Label:'Frodo'})
CALL gds.alpha.randomWalk.stream('undirected_interactions',
    {start:id(n), steps:5, walks:2}) 
YIELD nodeIds 
RETURN [nodeId in nodeIds | gds.util.asNode(nodeId).Label] as result

"""

read_query(random_walk)

,result
0,"[Frodo, Gimli, Thráin, Gandalf, Celeborn, Arag..."
1,"[Frodo, Arathorn, Isildur, Boromir, Merry, Den..."


You will get different results as it is a random walk algorithm after all.

### Triangle count and clustering coefficient

Another useful algorithm for analyzing social networks is Triangle Counting and Clustering Coefficient algorithm. A triangle composes of three nodes, where each node has a relationship to the other two. The clustering coefficient is a measure of the degree to which nodes in a graph tend to cluster together. This allows us to estimate how tightly-knit nodes in our graph are.

In [16]:
triangle_count = """

CALL gds.alpha.triangleCount.write('undirected_interactions', 
    {relationshipTypes:['INTERACTS_1'], writeProperty:'triangles', clusteringCoefficientProperty:'clustering'}) 
YIELD nodeCount, triangleCount, averageClusteringCoefficient

"""

read_query(triangle_count)

,nodeCount,triangleCount,averageClusteringCoefficient
0,44,803,0.542057


The global or average clustering coefficient is 0.54, which means that the persons in our graph are quite tightly-knit. We can also look at individuals and their local clustering coefficients.

In [17]:
local_clustering = """

MATCH (p:Person) 
RETURN p.Label as person,
       p.clustering as coefficient,
       p.triangles as triangles 
ORDER BY coefficient DESC LIMIT 5

"""

read_query(local_clustering)

,person,coefficient,triangles
0,Thráin,1.000000,6
1,Treebeard,1.000000,1
2,Goldberry,1.000000,3
3,Gildor,0.933333,14
4,Bill,0.857143,24


In [18]:
drop_graph('undirected_interactions')

### Categorical PageRank

Up until this point, all the above graph analysis could be done with the native projection. Cypher projection can project graphs that exist only at query time or when we want to use more advanced filtering than just node labels or relationship types. 

Categorical PageRank is a concept first introduced by Kenny Bastani in his blog post. I have also written a blog post about it using the Graph algorithms library. Now it is time to demonstrate it with the Graph Data Science library as well.

The idea behind it is pretty simple. As shown in the example above we have a graph of pages that have links between each other and might also belong to one or more categories. To better understand the global pagerank score of nodes in a network, we can breakdown our graph into several subgraphs, one for each category and execute the pagerank algorithm on each of that subgraphs. We store results as a relationship property between category and pages. This way we can break down which are the contributing categories to page’s global pagerank score.

We will start by assuming that each interaction is a positive endorsement(I know it's actually not, but let's pretend). We will breakdown our graph into several subgraphs by the class(men, elves), that the characters belong to. For example, when calculating pagerank for the category of men, all nodes will be considered, but only relationships that come from characters that belong to the class of men will be considered.

In [19]:
categorical_graph = """

CALL gds.graph.create.cypher( 'categorical_men', 
    // nodeQuery 
    'MATCH (n:Person) RETURN id(n) AS id 
     UNION 
    MATCH (n:Thing) RETURN id(n) as id',
    // relationshipQuery 
    'MATCH (c:Class)<-[:PART_OF]-(n)-[r:INTERACTS_1|:INTERACTS_2|:INTERACTS_3]-(m) 
    // Use the parameter
    WHERE c.id = $class 
    RETURN id(n) AS source, id(m) AS target, r.weight as weight,type(r) as type', 
    {parameters: { class: 'men' }})

"""

run_query(categorical_graph)

Let us now run the weighted pageRank on this graph.

In [20]:
weighted_pagerank = """

CALL gds.pageRank.stream('categorical_men',
    {relationshipWeightProperty:'weight'})
YIELD nodeId, score 
RETURN gds.util.asNode(nodeId).Label as name, score 
ORDER BY score DESC LIMIT 5

"""

read_query(weighted_pagerank)

,name,score
0,Gandalf,0.490396
1,Aragorn,0.472205
2,Frodo,0.321637
3,Pippin,0.311533
4,Théoden,0.307829


#### Weighted pagerank
To let know the algorithm that it should take relationship weights into account, we need to use relationshipWeightProperty parameter.

In [16]:
weigted_pagerank = """

CALL gds.pageRank.stream('undirected_weighted', {relationshipWeightProperty:'weight'}) 
YIELD nodeId, score 
RETURN gds.util.asNode(nodeId).Label as name, score 
ORDER BY score DESC LIMIT 5

"""

read_query(weigted_pagerank)

,name,score
0,Frodo,5.101857
1,Gandalf,3.757264
2,Sam,3.470734
3,Aragorn,3.246119
4,Pippin,2.355584


Gandalf comes out on top, with Aragorn following very closely in the second place. I am wondering if Aragorn has the most support from men in the third book, as he becomes their king at the end.

In [21]:
weighted_pagerank_third_book = """

CALL gds.pageRank.stream('categorical_men', 
    {relationshipTypes:['INTERACTS_3'],
     relationshipWeightProperty:'weight'})
YIELD nodeId, score 
RETURN gds.util.asNode(nodeId).Label as name, score 
ORDER BY score DESC LIMIT 5

"""

read_query(weighted_pagerank_third_book)

,name,score
0,Aragorn,0.517452
1,Gandalf,0.462588
2,Pippin,0.423120
3,Faramir,0.397746
4,Éomer,0.367025


As predicted, Aragorn takes the lead. Frodo is no longer on the list as he is quite isolated from everybody in the third book and walks alone with Sam to Mount Doom. To be honest, if you have Sam by your side, you are never lonely though.

In [22]:
drop_graph('categorical_men')

### Virtual categorical graph

We have only looked at the class of men and calculated the categorical pagerank for that specific subgraph. It would be very time consuming if we projected a subgraph for each class separately. That is why we will project a subgraph for each class in a single named graph using virtual relationship types. In the relationship query we have the option to return whatever we feel like as the column type. We will return the original relationship type combined with the class of the person to create virtual relationship types. This will allow us to calculate the categorical pagerank for each class.

In [24]:
virtual_graph = """

CALL gds.graph.create.cypher('categorical_virtual',
    'MATCH (n:Person) RETURN id(n) AS id 
     UNION MATCH (n:Thing) 
     RETURN id(n) as id', 
    'MATCH (c:Class)<-[:PART_OF]-(n)-[r:INTERACTS_1|:INTERACTS_2|:INTERACTS_3]-(m) 
     RETURN id(n) AS source, id(m) AS target, type(r) + c.id as type, r.weight as weight')

"""

run_query(virtual_graph)

We can now calculate the categorical pagerank for the class of elves.

In [25]:
categorical_elves = """

CALL gds.pageRank.stream('categorical_virtual',
    {relationshipTypes:['INTERACTS_1elves','INTERACTS_2elves','INTERACTS_3elves'],
     relationshipWeightProperty:'weight'})
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).Label as name, score
ORDER BY score DESC LIMIT 5

"""

read_query(categorical_elves)

,name,score
0,Frodo,0.353865
1,Aragorn,0.316407
2,Gandalf,0.251720
3,Elrond,0.250256
4,Galadriel,0.240393


If we want to calculate categorical pagerank for each class and store the results, we can use the same approach as in the original blog post, where we store the results in the relationship between a class and a person.

In [26]:
store_categorical = """

MATCH (c:Class) 
CALL gds.pageRank.stream('categorical_virtual',
    {relationshipTypes:['INTERACTS_1'+c.id,'INTERACTS_2'+c.id,'INTERACTS_3'+c.id],
     relationshipWeightProperty:'weight'})
YIELD nodeId, score 
WITH c, gds.util.asNode(nodeId) as node, score 
WHERE score > 0.151 
CREATE (c)-[:PAGERANK{score:score}]->(node)

"""

run_query(store_categorical)

We can now get the top three members for each class based on their categorical pagerank score.

In [27]:
top3_by_category = """

MATCH (c:Class)-[s:PAGERANK]->(p:Person)
WITH c, p, s.score as pagerank 
ORDER BY pagerank DESC 
RETURN c.id as class,
       collect(p.Label)[..3] as top_3_members

"""

read_query(top3_by_category)

,class,top_3_members
0,hobbit,"[Frodo, Sam, Gandalf]"
1,men,"[Gandalf, Aragorn, Frodo]"
2,elves,"[Frodo, Aragorn, Gandalf]"
3,dwarf,"[Gandalf, Gimli, Legolas]"
4,ainur,"[Frodo, Bombadil, Gandalf]"
5,animal,"[Sam, Gandalf, Frodo]"
6,orcs,"[Sam, Frodo, Shelob]"
7,thing,"[Frodo, Gandalf, Sam]"
8,ents,"[Gandalf, Merry, Pippin]"
